### PROJECT
### CONSUMER FINANCIAL COMPLAINTS AND RESPONSE ANALYTICS

#### PROBLEM STATEMENT
##### Financial institutions are required to respond to consumer complaints in a timely and effective manner to comply with regulatory standards set by the Consumer Financial Protection Bureau (CFPB).
##### The objective of this project is to analyze consumer financial complaints data to:
##### 1. Monitor complaint volume and trends over time
##### 2. Evaluate company responsiveness and resolution quality
##### 3. Identify high-risk financial products and issue types
#### The insights from this analysis can help financial institutions and fintech companies improve operational performance, reduce regulatory risk, and enhance customer experience.

#### This notebook contains:
##### 1. Importing the dataset and the required libraries, understading what each column of the dataset means, doing all the basic checks like checking for duplicates, datatypes od columns, missing values, etc.
##### 2. Data cleaning
##### 3. Feature Engineering
##### 4. Saving the dataset

In [1]:
# Importing the required libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Ignoring harmless warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Importing the raw dataset for analysis
df = pd.read_csv('../data/raw/raw_data.csv')
df.head()

,date_received,product,subproduct,issue,subissue,company_name,state,company_response_to_consumer,timely_response,consumer_disputed
0,2021-03-13,Debt collection,Other debt,Attempts to collect debt not owed,Debt was paid,ERC,IA,Closed with explanation,True,NaN
1,2019-12-31,Debt collection,Other debt,Attempts to collect debt not owed,Debt is not yours,ERC,CA,Closed with explanation,True,NaN
2,2021-03-01,Debt collection,Credit card debt,Attempts to collect debt not owed,Debt is not yours,ERC,NY,Closed with explanation,True,NaN
3,2021-08-16,Debt collection,I do not know,Attempts to collect debt not owed,Debt is not yours,ERC,NY,Closed with explanation,True,NaN
4,2019-11-21,Debt collection,Other debt,Attempts to collect debt not owed,Debt is not yours,ERC,PA,Closed with explanation,True,NaN


#### Understanding the meaning of each column:
##### date_received: The date on which the consumer complaint was received by the CFPB.
##### product: The financial product or service against which the complaint is filed.
##### subproduct: Specific subproduct of the product category.
##### issue: The complaint that has been filed against the project.
##### subissue: Detailed complaint.
##### company_name: The name of the company that owns the financial product or service.
##### state: The state to which the company belongs to.
##### company_response_to_consumer: The company's response to the consumer's complaint.
##### timely_response: Whether the company has responded to the complaint in the given time.
##### consumer_disputed: Whether the consumer disagreed with the company's response.

#### UNDERSTANDING THE DATA - DOING BASIC CHECKS

In [3]:
# Checking the number of rows and columns in the dataset:

print(f"The number of rows in the dataset is {df.shape[0]} and the number of columns is {df.shape[1]}.")

The number of rows in the dataset is 2262724 and the number of columns is 10.


In [4]:
# Checking for duplicates in the dataset:
print(f"The number of duplicate rows in the dataset is {df.duplicated().sum()}.")
print(f"The percentage of duplicate rows in the dataset is {df.duplicated().mean()*100} %.")

The number of duplicate rows in the dataset is 831228.
The percentage of duplicate rows in the dataset is 36.73572207657673 %.


In [5]:
# Removing duplicate rows of the dataset:
df = df.drop_duplicates()

In [6]:
# Checking if the duplicates are removed successfully
print(f"After removing duplicates, the number of rows in the dataset is {df.shape[0]}.")
print(f"The number of duplicate rows in the dataset is now {df.duplicated().sum()}.")

After removing duplicates, the number of rows in the dataset is 1431496.
The number of duplicate rows in the dataset is now 0.


In [7]:
# Checking the datatype of each column:

df.dtypes

date_received                    object
product                          object
subproduct                       object
issue                            object
subissue                         object
company_name                     object
state                            object
company_response_to_consumer     object
timely_response                    bool
consumer_disputed               float64
dtype: object

##### The datatype of date_received is object, it has to be converted to datetime.

In [8]:
# Checking for the percentage of missing values in each column

df.isnull().mean()*100

date_received                     0.000000
product                           0.000000
subproduct                        0.007335
issue                             0.000000
subissue                         12.149458
company_name                      0.000000
state                             1.510238
company_response_to_consumer      0.000210
timely_response                   0.000000
consumer_disputed               100.000000
dtype: float64

In [9]:
# Checking for the number of missing values in each column

df.isnull().sum()

date_received                         0
product                               0
subproduct                          105
issue                                 0
subissue                         173919
company_name                          0
state                             21619
company_response_to_consumer          3
timely_response                       0
consumer_disputed               1431496
dtype: int64

##### The columns subproduct and company_response_to_consumer have very few missing values. So those particular rows can be dropped. The missing values in the columns state and subissue can be filled by their mode values. The column consumer_disputed has all missing values, so the entire column needs to be dropped.

In [10]:
# Checking the number of unique values in each column

df.nunique()

date_received                   1542
product                            9
subproduct                        48
issue                             81
subissue                         166
company_name                    5246
state                             61
company_response_to_consumer       5
timely_response                    2
consumer_disputed                  0
dtype: int64

##### Except for product, company_response_to_consumer, and timely_response, all other columns have high cardinality.

In [11]:
# Finding the unique values in the columns 'product', 'company_response_to_consumer', and 'timely_response':
print(f"The unique values in the 'product' column are: {df['product'].unique()}")
print(f"The unique values in the 'company_response_to_consumer' column are: {df['company_response_to_consumer'].unique()}")
print(f"The unique values in the 'timely_response' column are: {df['timely_response'].unique()}")

The unique values in the 'product' column are: ['Debt collection'
 'Credit reporting, credit repair services, or other personal consumer reports'
 'Money transfer, virtual currency, or money service' 'Student loan'
 'Mortgage' 'Checking or savings account' 'Credit card or prepaid card'
 'Payday loan, title loan, or personal loan' 'Vehicle loan or lease']
The unique values in the 'company_response_to_consumer' column are: ['Closed with explanation' 'In progress' 'Closed with monetary relief'
 'Closed with non-monetary relief' 'Untimely response' nan]
The unique values in the 'timely_response' column are: [ True False]


#### DATA CLEANING
##### CONVERTING THE DTYPE OF THE 'date_received' COLUMN TO DATETIME:

In [12]:
df["date_received"] = pd.to_datetime(df["date_received"])

# Checking if the conversion is done correctly:
df.dtypes

date_received                   datetime64[ns]
product                                 object
subproduct                              object
issue                                   object
subissue                                object
company_name                            object
state                                   object
company_response_to_consumer            object
timely_response                           bool
consumer_disputed                      float64
dtype: object

##### HANDLING MISSING VALUES

In [13]:
# Dropping the column "consumer_disputed", since all of its values were missing:

df.drop("consumer_disputed", axis=1, inplace=True)

In [14]:
# Dropping the rows with missing values in the 'company_response_to_customer' and 'subproduct'columns:
df = df.dropna(subset=['company_response_to_consumer', 'subproduct'])

# Checking if they have been removed successfully:
df.isnull().sum()

date_received                        0
product                              0
subproduct                           0
issue                                0
subissue                        173915
company_name                         0
state                            21614
company_response_to_consumer         0
timely_response                      0
dtype: int64

In [15]:
# The missing values in the 'state' can be imputed using the mode of the column.
# Finding the mode of the 'state' column:
state_mode = df['state'].mode()[0]
state_mode

'CA'

In [16]:
# Filling the missing values in the 'state' column with the mode:
df['state'] = df['state'].fillna(state_mode)

# Checking if the missing values in the 'state' column have been filled successfully:
df.isnull().sum()

date_received                        0
product                              0
subproduct                           0
issue                                0
subissue                        173915
company_name                         0
state                                0
company_response_to_consumer         0
timely_response                      0
dtype: int64

In [ ]:
# The missing values in the 'subissue' column needs to be handled using a different strategy.
# Each subissue is a subcategory of a broader category 'issue'. So, instead of blindly filling the missing values
# using the mode of the entire 'subissue' column, we can find the mode of 'subissue' for each category in the 'issue' column.
# First finding the rows with missing values in the 'subissue' column:
missing_subissue = df[df['subissue'].isnull()]
missing_subissue

,date_received,product,subproduct,issue,subissue,company_name,state,company_response_to_consumer,timely_response
40,2022-01-07,"Money transfer, virtual currency, or money ser...",Mobile or digital wallet,"Managing, opening, or closing your mobile wall...",NaN,Abra,CA,Closed with explanation,True
75,2021-11-05,Mortgage,Conventional home mortgage,Closing on a mortgage,NaN,KEYCORP,OH,Closed with explanation,True
76,2020-03-11,Mortgage,Conventional home mortgage,Trouble during payment process,NaN,KEYCORP,CT,Closed with explanation,True
77,2021-06-16,Mortgage,Other type of mortgage,Trouble during payment process,NaN,KEYCORP,MA,Closed with explanation,True
78,2020-12-01,Mortgage,Home equity loan or line of credit (HELOC),Applying for a mortgage or refinancing an exis...,NaN,KEYCORP,NY,Closed with explanation,True
...,...,...,...,...,...,...,...,...,...
2262667,2019-11-20,Mortgage,Conventional home mortgage,Trouble during payment process,NaN,"PEOPLE'S UNITED BANK, NATIONAL ASSOCIATION",CT,Closed with explanation,True
2262690,2021-05-17,Mortgage,Conventional home mortgage,Applying for a mortgage or refinancing an exis...,NaN,CALCON MUTUAL MORTGAGE LLC DBA ONETRUST HOME L...,NY,Closed with non-monetary relief,False
2262713,2019-10-28,Mortgage,Conventional home mortgage,Struggling to pay mortgage,NaN,"Community Loan Servicing, LLC (formerly known ...",TN,Closed with explanation,True
2262714,2022-02-22,Mortgage,Conventional home mortgage,Applying for a mortgage or refinancing an exis...,NaN,"Community Loan Servicing, LLC (formerly known ...",OK,Closed with explanation,True


In [18]:
# Finding the categories in the 'issue' column in the above dataset 'missing_subissue':
missing_subissue['issue'].value_counts()

issue
Trouble during payment process                                                      47723
Struggling to pay mortgage                                                          23859
Fraud or scam                                                                       16371
Applying for a mortgage or refinancing an existing mortgage                         15646
Closing on a mortgage                                                                9351
Problem with fraud alerts or security freezes                                        9161
Other transaction problem                                                            6881
Managing, opening, or closing your mobile wallet account                             4520
Charged fees or interest you didn't expect                                           4427
Unauthorized transactions or other transaction problem                               4030
Money was not available when promised                                                3977
Unex

In [ ]:
# Finding the mode value of each category in the 'subissue' column corresponding to each category in the 'issue' column:

for cat in missing_subissue["issue"].unique():
    print(df.groupby("issue").agg({'subissue': pd.Series.mode}).loc[cat])

subissue    []
Name: Managing, opening, or closing your mobile wallet account, dtype: object
subissue    []
Name: Closing on a mortgage, dtype: object
subissue    []
Name: Trouble during payment process, dtype: object
subissue    []
Name: Applying for a mortgage or refinancing an existing mortgage, dtype: object
subissue    []
Name: Unexpected or other fees, dtype: object
subissue    []
Name: Charged fees or interest you didn't expect, dtype: object
subissue    []
Name: Getting the loan, dtype: object
subissue    []
Name: Problem with fraud alerts or security freezes, dtype: object
subissue    []
Name: Struggling to pay mortgage, dtype: object
subissue    []
Name: Fraud or scam, dtype: object
subissue    []
Name: Unauthorized transactions or other transaction problem, dtype: object
subissue    []
Name: Problem with customer service, dtype: object
subissue    []
Name: Other transaction problem, dtype: object
subissue    []
Name: Was approved for a loan, but didn't receive the money, dty

##### From the above series, it is observed that the mode of the most of the subissue category is nan. So, we may have to impute these missing values with the mode of the entire column. But it may introduce wrong values and reduce the data quality. So, it is better to drop the entire 'subissue' column, since we already have the 'issue' column which can be used for analysis.

In [26]:
# Dropping the subissue column:
df = df.drop("subissue", axis=1)
df.head()

,date_received,product,subproduct,issue,company_name,state,company_response_to_consumer,timely_response
0,2021-03-13,Debt collection,Other debt,Attempts to collect debt not owed,ERC,IA,Closed with explanation,True
1,2019-12-31,Debt collection,Other debt,Attempts to collect debt not owed,ERC,CA,Closed with explanation,True
2,2021-03-01,Debt collection,Credit card debt,Attempts to collect debt not owed,ERC,NY,Closed with explanation,True
3,2021-08-16,Debt collection,I do not know,Attempts to collect debt not owed,ERC,NY,Closed with explanation,True
4,2019-11-21,Debt collection,Other debt,Attempts to collect debt not owed,ERC,PA,Closed with explanation,True


##### FEATURE ENGINEERING
##### Creating new features "month_and_year" denoting the month and year in which the complaint was received by the CFPB, and "year" denoting the year in which the complaint was received.

In [29]:
df['month_and_year'] = df['date_received'].dt.to_period('M')

In [ ]:
df.head()

,date_received,product,subproduct,issue,company_name,state,company_response_to_consumer,timely_response,month_and_year
0,2021-03-13,Debt collection,Other debt,Attempts to collect debt not owed,ERC,IA,Closed with explanation,True,2021-03
1,2019-12-31,Debt collection,Other debt,Attempts to collect debt not owed,ERC,CA,Closed with explanation,True,2019-12
2,2021-03-01,Debt collection,Credit card debt,Attempts to collect debt not owed,ERC,NY,Closed with explanation,True,2021-03
3,2021-08-16,Debt collection,I do not know,Attempts to collect debt not owed,ERC,NY,Closed with explanation,True,2021-08
4,2019-11-21,Debt collection,Other debt,Attempts to collect debt not owed,ERC,PA,Closed with explanation,True,2019-11


In [32]:
df["year_received"] = df['date_received'].dt.year

In [35]:
# Since 'year' and 'month_and_year' columns are created from the 'date_received' column, we can drop the 'date_received' column now.
df = df.drop("date_received", axis=1)

In [36]:
# Checking the final dataset:

df.head()

,product,subproduct,issue,company_name,state,company_response_to_consumer,timely_response,month_and_year,year_received
0,Debt collection,Other debt,Attempts to collect debt not owed,ERC,IA,Closed with explanation,True,2021-03,2021
1,Debt collection,Other debt,Attempts to collect debt not owed,ERC,CA,Closed with explanation,True,2019-12,2019
2,Debt collection,Credit card debt,Attempts to collect debt not owed,ERC,NY,Closed with explanation,True,2021-03,2021
3,Debt collection,I do not know,Attempts to collect debt not owed,ERC,NY,Closed with explanation,True,2021-08,2021
4,Debt collection,Other debt,Attempts to collect debt not owed,ERC,PA,Closed with explanation,True,2019-11,2019


In [37]:
# Checking for missing values and dtypes once again:
df.isnull().sum(), df.dtypes

(product                         0
 subproduct                      0
 issue                           0
 company_name                    0
 state                           0
 company_response_to_consumer    0
 timely_response                 0
 month_and_year                  0
 year_received                   0
 dtype: int64,
 product                            object
 subproduct                         object
 issue                              object
 company_name                       object
 state                              object
 company_response_to_consumer       object
 timely_response                      bool
 month_and_year                  period[M]
 year_received                       int32
 dtype: object)

##### SAVING THE CLEANED DATASET

In [38]:
# Saving the cleaned dataset to the processed folder in the data directory:
df.to_csv('../data/processed/cleaned_data.csv', index=False)